<a href="https://colab.research.google.com/github/dienyffer2024/Bucket/blob/main/Projeto_Bucket_Tratamento_Manipulacao_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalar bilbiotecas e Autenticar a conta Google Cloud

In [1]:
!pip install pandas google-cloud-storage mysql-connector-python

import pandas as pd
from google.cloud import storage
import mysql.connector
from mysql.connector import Error

# Autenticar a conta do Google Cloud
from google.colab import auth
auth.authenticate_user()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 10.8 MB/s eta 0:00:00


# Importando o arquivo e convertendo para CSV

In [ ]:
excel_file_path = '/balanco_petro.xlsx'

csv_file_path = '/balanco_petro.csv'

# Ler o arquivo Excel
df = pd.read_excel(excel_file_path)

# Salvar como CSV
df.to_csv(csv_file_path, index=False)

print(f'Arquivo Excel {excel_file_path} convertido para CSV {csv_file_path}')

Arquivo Excel /balanco_petro.xlsx convertido para CSV /balanco_petro.csv


# Configurar cliente do Google Cloud Storage e fazer upload para o Bucket

In [ ]:
client = storage.Client()
bucket_name = 'petrobras'
file_path = '/balanco_petro.csv'

def upload_to_gcs(bucket_name, file_path):
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob('dados_brutos/arquivo.csv')
    blob.upload_from_filename(file_path)
    print(f'Arquivo {file_path} carregado para {bucket_name}/dados_brutos/arquivo.csv')

upload_to_gcs(bucket_name, file_path)

Arquivo /balanco_petro.csv carregado para petrobras/dados_brutos/arquivo.csv


# Definindo nome do Bucket, fazer o download do Dataframe como bytes

In [ ]:
from google.cloud import storage
import pandas as pd
import io
import csv

client = storage.Client()

bucket_name = 'petrobras'
file_name = 'dados_brutos/arquivo.csv'

# Obtendo o bucket e o blob (arquivo)
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_name)

# Download do arquivo como bytes
data = blob.download_as_bytes()

try:
    df = pd.read_csv(io.BytesIO(data), delimiter=';', encoding='latin-1', quoting=csv.QUOTE_NONE, on_bad_lines='skip')
    print(df.head())
except pd.errors.ParserError as e:
    print(f"Erro ao ler o arquivo CSV: {e}")
except Exception as e:
    print(f"Erro inesperado: {e}")

  "XLSWrite 1.34 Copyright(c) 1999,2000 Axolot Data",BalanÃ§o Patrimonial - PETROBRAS,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60
0  31/03/2009,30/06/2009,30/09/2009,31/12/2009,31...                                                                                                                                                                  

#Criando um diretório no Bucket

In [ ]:
import os

bucket_name = 'petrobras'

def read_csv_from_gcs(bucket_name, file_name):
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)

    os.makedirs(os.path.dirname(file_name), exist_ok=True)

    blob.download_to_filename(file_name)
    df = pd.read_csv(file_name)
    return df

df = read_csv_from_gcs(bucket_name, 'dados_brutos/arquivo.csv')
print(df.head())

  XLSWrite 1.34 Copyright(c) 1999,2000 Axolot Data  \
0                                       31/03/2009   
1                                      Ativo Total   
2                                 Ativo Circulante   
3                    Caixa e Equivalentes de Caixa   
4                           Aplicações Financeiras   

  Balanço Patrimonial - PETROBRAS     Unnamed: 2     Unnamed: 3    Unnamed: 4  \
0                      30/06/2009     30/09/2009     31/12/2009    31/03/2010   
1                   305265344.512  333789528.064  345607274.496     365998080   
2                    57621536.768   75719057.408   76674015.232  74459103.232   
3                    10072162.304   30088286.208   28795713.536  26951325.696   
4                               0              0              0             0   

      Unnamed: 5     Unnamed: 6     Unnamed: 7     Unnamed: 8     Unnamed: 9  \
0     30/06/2010     30/09/2010     31/12/2010     31/03/2011     30/06/2011   
1  382029856.768  507697037.

# Tratamento dos dados inconsistentes

- Alterar a primeira coluna para "Data"

In [ ]:
df.iat[0, 0] = 'Data'

print(df)

     XLSWrite 1.34 Copyright(c) 1999,2000 Axolot Data  \
0                                                Data   
1                                         Ativo Total   
2                                    Ativo Circulante   
3                       Caixa e Equivalentes de Caixa   
4                              Aplicações Financeiras   
5                                    Contas a Receber   
6                                            Estoques   
7                                   Ativos Biológicos   
8                                Tributos a Recuperar   
9                                Despesas Antecipadas   
10                          Outros Ativos Circulantes   
11                     Ativo Realizável a Longo Prazo   
12     Aplicações Financeiras Avaliadas a Valor Justo   
13  Aplicações Financeiras Avaliadas ao Custo Amor...   
14                                   Contas a Receber   
15                                           Estoques   
16                             

- Redefinindo a coluna para valor numérico

In [ ]:
df.columns = range(df.shape[1])

print(df)

                                                   0              1   \
0                                                Data     30/06/2009   
1                                         Ativo Total  305265344.512   
2                                    Ativo Circulante   57621536.768   
3                       Caixa e Equivalentes de Caixa   10072162.304   
4                              Aplicações Financeiras              0   
5                                    Contas a Receber   14555268.096   
6                                            Estoques   19674548.224   
7                                   Ativos Biológicos              0   
8                                Tributos a Recuperar              0   
9                                Despesas Antecipadas              0   
10                          Outros Ativos Circulantes   13319559.168   
11                     Ativo Realizável a Longo Prazo   24442390.528   
12     Aplicações Financeiras Avaliadas a Valor Justo           

- Transpor os dados para melhorar a visualização

In [ ]:
df_transposto = df.T
display(df_transposto)

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,Data,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber,Estoques,Ativos Biológicos,Tributos a Recuperar,Despesas Antecipadas,...,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital
1,30/06/2009,305265344.512,57621536.768,10072162.304,0,14555268.096,19674548.224,0,0,0,...,149847965.696,78966685.696,514856.992,9920,58865377.28,10918406.144,572716.032,0,0,0
2,30/09/2009,333789528.064,75719057.408,30088286.208,0,13643311.104,20635084.8,0,0,0,...,155391688.704,78966685.696,514856.992,9718,58866671.616,16466363.392,567398.976,0,0,0
3,31/12/2009,345607274.496,76674015.232,28795713.536,0,13984270.336,21424652.288,0,0,0,...,159464587.264,78966685.696,514856.992,350,79521013.76,0,461687.008,0,0,0
4,31/03/2010,365998080,74459103.232,26951325.696,0,16200354.816,20030609.408,0,0,0,...,170299080.704,78966685.696,514856.992,0,84879695.872,5971459.072,-33617,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,31/03/2023,978576998.4,157193994.24,52276998.144,14629000.192,23497000.96,40483000.32,0,6102000.128,0,...,402083980.288,205431996.416,3102000.128,0,166717997.056,0,0,0,26831998.976,0
57,30/06/2023,990459002.88,136816001.024,49882001.408,11102999.552,21041000.448,39240998.912,0,6352999.936,0,...,370682983.552,205431996.416,3102000.128,0,135016988.672,0,0,0,27132000.256,0
58,30/09/2023,1025495990.272,147311001.6,60642000.896,6504999.936,25501999.104,39509999.616,0,5965000.192,0,...,386006988.544,205431996.416,2131000.064,0,146422005.76,0,0,0,32021999.616,0
59,31/12/2023,1050887979.008,157079011.328,61612998.656,13649999.872,29702000.64,37184000,0,5703000.064,0,...,380441005.824,205431996.416,-538000,0,159171002.368,0,0,0,16376000.512,0


# Salvando a tabela com os tratamentos realizados

In [ ]:
from google.colab import files

csv_file_path = 'tabela_tratada.csv'
df_transposto.to_csv(csv_file_path, index=False)

# Fazer o download do arquivo CSV
files.download(csv_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Fazendo o upload do Dataframe com correções para o Bucket

In [ ]:
client = storage.Client()
bucket_name = 'petrobras'
file_path = '/tabela_tratada.csv'

def upload_to_gcs(bucket_name, file_path):
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob('dados_tratados/arquivo.csv')
    blob.upload_from_filename(file_path)
    print(f'Arquivo {file_path} carregado para {bucket_name}/dados_tratados/arquivo.csv')

# Fazer upload do arquivo para o bucket GCS
upload_to_gcs(bucket_name, file_path)

Arquivo /tabela_tratada.csv carregado para petrobras/dados_tratados/arquivo.csv


# Definindo as credenciais e informações da conexão MySQL

In [ ]:
servidor = '34.46.115.184'
basededados = 'petrobras'
usuario = 'bd-soulcode'
senha = 'JFRkso251@'

def conectar_mysql(servidor, basededados, usuario, senha):
    try:
        conn = mysql.connector.connect(
            host=servidor,
            user=usuario,
            password=senha
        )
        if conn.is_connected():
            print("Conexão bem-sucedida ao MySQL")
            return conn
    except Error as e:
        print(f"Erro ao conectar ao MySQL: {e}")
        return None

conexao = conectar_mysql(servidor, basededados, usuario, senha)

Conexão bem-sucedida ao MySQL


# Função para criar o banco de dados

In [ ]:
def criar_banco(conexao, basededados):
    try:
        cursor = conexao.cursor()
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {basededados}")
        conexao.commit()
        print(f"Banco de dados {basededados} criado ou já existente.")
    except Error as e:
        print(f"Erro ao criar banco de dados: {e}")

criar_banco(conexao, basededados)

Banco de dados petrobras criado ou já existente.


# Renomear as colunas para nomes válidos

In [ ]:
df.columns = [f"col_{i}" for i in range(df.shape[1])]

# Criar tabela através de Conexão no GCP

In [ ]:
def criar_tabela(conexao, df):
    cursor = conexao.cursor()
    columns = ', '.join(f"{col} VARCHAR(255)" for col in df.columns)
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS balanço (
        id INT AUTO_INCREMENT PRIMARY KEY,
        {columns}
    )
    """
    try:
        cursor.execute(create_table_query)
        conexao.commit()
        print("Tabela criada com sucesso.")
    except Error as e:
        print(f"Erro ao criar tabela: {e}")

import mysql.connector
from mysql.connector import Error

try:
    conexao = mysql.connector.connect(
        host='34.46.115.184',
        user='bd-soulcode',
        password='Dieny@1993',
        database='petrobras'
    )
    if conexao.is_connected():
        criar_tabela(conexao, df)
except Error as e:
    print(f"Erro ao conectar ao MySQL: {e}")
finally:
    if conexao.is_connected():
        conexao.close()
        print("Conexão ao MySQL encerrada.")


Tabela criada com sucesso.
Conexão ao MySQL encerrada.


# Função para conectar ao banco de dados

In [ ]:
import mysql.connector
from mysql.connector import Error

def conectar_ao_banco():
    try:
        conexao = mysql.connector.connect(
            host='34.46.115.184',
            user='bd-soulcode',
            password='Dieny@1993',
            database='petrobras'
        )
        if conexao.is_connected():
            print("Conectado ao banco de dados.")
            return conexao
    except Error as e:
        print(f"Erro ao conectar ao MySQL: {e}")
        return None

# Função para inserir dados do DataFrame na tabela
def inserir_dados(df, conexao):
    cursor = conexao.cursor()
    for _, row in df.iterrows():
        values = ', '.join(f"'{str(val)}'" for val in row)
        insert_query = f"INSERT INTO balanço ({', '.join(df.columns)}) VALUES ({values})"
        try:
            cursor.execute(insert_query)
        except Error as e:
            print(f"Erro ao inserir dados: {e}")
    conexao.commit()
    print("Dados inseridos com sucesso.")

# Conectar ao banco de dados e inserir dados
conexao = conectar_ao_banco()
if conexao:
    inserir_dados(df, conexao)
    conexao.close()


Conectado ao banco de dados.
Dados inseridos com sucesso.


# Função para consultar dados

In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

def consultar_dados(conexao):
    try:
        cursor = conexao.cursor()
        cursor.execute("USE petrobras")

        query = "SELECT * FROM balanço"
        df_resultado = pd.read_sql(query, con=conexao)
        print(df_resultado)
    except Error as e:
        print(f"Erro ao consultar dados: {e}")

conexao = conectar_ao_banco()
if conexao:
    consultar_dados(conexao)
    conexao.close()


Conectado ao banco de dados.
    id                                              col_0          col_1  \
0    1                                               Data     30/06/2009   
1    2                                        Ativo Total  305265344.512   
2    3                                   Ativo Circulante   57621536.768   
3    4                      Caixa e Equivalentes de Caixa   10072162.304   
4    5                             Aplicações Financeiras              0   
5    6                                   Contas a Receber   14555268.096   
6    7                                           Estoques   19674548.224   
7    8                                  Ativos Biológicos              0   
8    9                               Tributos a Recuperar              0   
9   10                               Despesas Antecipadas              0   
10  11                          Outros Ativos Circulantes   13319559.168   
11  12                     Ativo Realizável a Longo Prazo  

<ipython-input-90-a33b0f581045>:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultado = pd.read_sql(query, con=conexao)


# Fechar a conexão

In [ ]:
if conexao.is_connected():
    conexao.close()
    print("Conexão fechada.")